In [3]:
pip install nltk scikit-learn streamlit

   ---------------------------------------- 0.0/9.6 MB ? eta -:--:--
   - -------------------------------------- 0.3/9.6 MB ? eta -:--:--
   --- ------------------------------------ 0.8/9.6 MB 2.4 MB/s eta 0:00:04
   ---- ----------------------------------- 1.0/9.6 MB 1.9 MB/s eta 0:00:05
   ----- ---------------------------------- 1.3/9.6 MB 1.7 MB/s eta 0:00:05
   ------ --------------------------------- 1.6/9.6 MB 1.7 MB/s eta 0:00:05
   ------- -------------------------------- 1.8/9.6 MB 1.5 MB/s eta 0:00:05
   -------- ------------------------------- 2.1/9.6 MB 1.6 MB/s eta 0:00:05
   --------- ------------------------------ 2.4/9.6 MB 1.5 MB/s eta 0:00:05
   ---------- ----------------------------- 2.6/9.6 MB 1.5 MB/s eta 0:00:05
   ------------- -------------------------- 3.1/9.6 MB 1.5 MB/s eta 0:00:05
   -------------- ------------------------- 3.4/9.6 MB 1.5 MB/s eta 0:00:05
   ---------------- ----------------------- 3.9/9.6 MB 1.6 MB/s eta 0:00:04
   ----------------- -----


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import nltk
import random
import os
import ssl
import streamlit as st
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

In [10]:
ssl._create_default_https_context = ssl._create_unverified_context
nltk.data.path.append(os.path.abspath('nltk_data'))
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rohit\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [14]:
intents = [
    {
        "tag": "greeting",
        "patterns": ["Hi", "Hello", "Hey", "How are you", "What's up"],
        "responses": ["Hi there", "Hello", "Hey", "I'm fine, thank you", "Nothing much"]
    },
    {
        "tag": "goodbye",
        "patterns": ["Bye", "See you later", "Goodbye", "Take care"],
        "responses": ["Goodbye", "See you later", "Take care"]
    },
    {
        "tag": "thanks",
        "patterns": ["Thank you", "Thanks", "Thanks a lot", "I appreciate it"],
        "responses": ["You're welcome", "No problem", "Glad I could help"]
    },
    {
        "tag": "about",
        "patterns": ["What can you do", "Who are you", "What are you", "What is your purpose"],
        "responses": ["I am a chatbot", "My purpose is to assist you", "I can answer questions and provide assistance"]
    },
    {
        "tag": "help",
        "patterns": ["Help", "I need help", "Can you help me", "What should I do"],
        "responses": ["Sure, what do you need help with?", "I'm here to help. What's the problem?", "How can I assist you?"]
    },
    {
        "tag": "age",
        "patterns": ["How old are you", "What's your age"],
        "responses": ["I don't have an age. I'm a chatbot.", "I was just born in the digital world.", "Age is just a number for me."]
    },
    {
        "tag": "weather",
        "patterns": ["What's the weather like", "How's the weather today"],
        "responses": ["I'm sorry, I cannot provide real-time weather information.", "You can check the weather on a weather app or website."]
    },
    {
        "tag": "budget",
        "patterns": ["How can I make a budget", "What's a good budgeting strategy", "How do I create a budget"],
        "responses": [
            "To make a budget, start by tracking your income and expenses. Then, allocate your income towards essential expenses like rent, food, and bills. Next, allocate some of your income towards savings and debt repayment. Finally, allocate the remainder of your income towards discretionary expenses like entertainment and hobbies.",
            "A good budgeting strategy is to use the 50/30/20 rule. This means allocating 50% of your income towards essential expenses, 30% towards discretionary expenses, and 20% towards savings and debt repayment.",
            "To create a budget, start by setting financial goals for yourself. Then, track your income and expenses for a few months to get a sense of where your money is going. Next, create a budget by allocating your income towards essential expenses, savings and debt repayment, and discretionary expenses."
        ]
    },
    {
        "tag": "credit_score",
        "patterns": ["What is a credit score", "How do I check my credit score", "How can I improve my credit score"],
        "responses": [
            "A credit score is a number that represents your creditworthiness. It is based on your credit history and is used by lenders to determine whether or not to lend you money. The higher your credit score, the more likely you are to be approved for credit.",
            "You can check your credit score for free on several websites such as Credit Karma and Credit Sesame."
        ]
    },
    {
        "tag": "health",
        "patterns": ["How can I stay healthy?", "Give me health tips", "How to improve my health?"],
        "responses": [
            "Eat a balanced diet, exercise regularly, get enough sleep, and stay hydrated.",
            "Avoid processed foods, drink plenty of water, and get regular medical check-ups.",
            "Prioritize mental health along with physical fitness for overall well-being."
        ]
    },
    {
        "tag": "exercise",
        "patterns": ["What are some good exercises?", "How to stay fit?", "Give me a workout plan"],
        "responses": [
            "Try cardio exercises like running, cycling, and swimming.",
            "Strength training and yoga are great for building flexibility and endurance.",
            "Start with a mix of cardio and strength training for a balanced routine."
        ]
    },
    {
        "tag": "travel",
        "patterns": ["Where should I travel?", "Best travel destinations?", "Suggest a travel destination"],
        "responses": [
            "It depends on your preference! Do you like beaches, mountains, or cities?",
            "Paris, Bali, and Tokyo are popular travel destinations.",
            "For adventure, try New Zealand. For culture, visit Italy."
        ]
    },
    {
        "tag": "food",
        "patterns": ["What should I eat?", "Suggest a meal", "Best healthy foods?"],
        "responses": [
            "Try a healthy meal like grilled salmon with veggies.",
            "How about a smoothie bowl for a nutritious breakfast?",
            "Fruits, vegetables, nuts, and lean proteins are great for a healthy diet."
        ]
    },
    {
        "tag": "technology",
        "patterns": ["Tell me about the latest technology", "What is AI?", "What are the latest tech trends?"],
        "responses": [
            "AI, blockchain, and quantum computing are shaping the future.",
            "Artificial Intelligence is a technology that enables machines to learn and make decisions.",
            "Latest tech trends include AI, 5G, and edge computing."
        ]
    },
    {
        "tag": "entertainment",
        "patterns": ["What should I watch?", "Suggest a good movie", "Best TV shows?"],
        "responses": [
            "If you like action, try 'John Wick'. For sci-fi, watch 'Interstellar'.",
            "You might enjoy TV shows like 'Stranger Things' or 'Breaking Bad'.",
            "Try a mix of different genres to find what you enjoy!"
        ]
    },
    {
        "tag": "motivation",
        "patterns": ["Give me motivation", "Motivational quotes", "I need inspiration"],
        "responses": [
            "Believe in yourself and all that you are!",
            "Success is not final, failure is not fatal: it is the courage to continue that counts.",
            "Dream big, work hard, and make it happen."
        ]
    },
    {
        "tag": "books",
        "patterns": ["Suggest a good book", "What book should I read?", "Best books to read?"],
        "responses": [
            "If you like self-improvement, try 'Atomic Habits' by James Clear.",
            "'The Alchemist' by Paulo Coelho is a great inspirational book.",
            "For business, read 'The Lean Startup' by Eric Ries."
        ]
    },
    {
        "tag": "finance",
        "patterns": ["How to save money?", "Best investment tips?", "How to be financially stable?"],
        "responses": [
            "Save at least 20% of your income and invest wisely.",
            "Diversify your investments and avoid unnecessary expenses.",
            "Track your expenses and create a budget for financial stability."
        ]
    },
    {
        "tag": "career",
        "patterns": ["How to find a job?", "Career advice", "How to grow in my career?"],
        "responses": [
            "Build your skills, network with professionals, and apply for relevant jobs.",
            "Stay updated with industry trends and continuously learn new skills.",
            "Work on projects, gain experience, and improve your resume."
        ]
    }
]
